# Création de votre première campagne

Ce bloc-notes vous guide dans les étapes de la construction d'un modèle de recommandation de films basé sur les données collectées à partir de le jeu de données MovieLens. L'objectif est de recommander des films pertinents pour un utilisateur en particulier.

Les données proviennent du projet MovieLens. Vous pouvez en savoir plus sur ces données et leurs utilisations potentielles en effectuant une recherche sur le web pendant l'une des périodes d'attente dans les cellules ci-dessous.

## Comment utiliser le bloc-notes

Le code est décomposé en cellules comme celle ci-dessous. Il y a un bouton `Run` triangulaire en haut de cette page qui vous permet d'exécuter chaque cellule et de passer à la suivante, ou vous pouvez appuyer sur `Shift` + `Enter` lorsque vous êtes dans la cellule pour l'exécuter et passer à la suivante.

Lorsqu'une cellule est en cours d'exécution, vous remarquerez une ligne sur le coté affichant une `*` pendant que la cellule est opérationnelle ou elle se mettra à jour sous forme de nombre pour indiquer la dernière cellule qui a terminé l'exécution après avoir achevé l'exécution de tous les codes dans une cellule.


Il suffit de suivre les instructions ci-dessous et d'exécuter les cellules pour commencer avec Amazon Personalize.

## Importations 

Python est livré avec une large gamme de bibliothèques. Nous devons les importer, ainsi que celles qui sont installées pour nous aider, comme [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK pour python) et [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), qui sont des outils essentiels pour la science des données.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
!conda install -y -c conda-forge unzip

Ensuite, vous devez vérifier que votre environnement peut communiquer avec Amazon Personalize. Les lignes ci-dessous servent à cela.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Configurer les données

Les données sont importées dans Amazon Personalize par le biais d'Amazon S3. Nous allons spécifier ci-dessous un compartiment que vous avez créé dans AWS pour les besoins de cet exercice.

Vous allez mettre à jour la variable `bucket` ci-dessous pour qu'elle ait la valeur que vous avez créée plus tôt dans les étapes de CloudFormation, et qui doit se trouver dans un fichier texte de votre travail antérieur. Le `filename` n'a pas besoin d'être modifié.

### Spécifier un compartiment et un emplacement de sortie des données
Veillez à mettre à jour la valeur `bucket` si vous l'avez personnalisée pendant le déploiement du modèle CloudFormation. Cliquez dans la cellule ci-dessous pour effectuer des modifications.

In [ ]:
bucket = "personalizedemofirstnamelastname"       # replace with the name of your S3 bucket
filename = "movie-lens-100k.csv"

### Télécharger, préparer et charger les données d'entraînement

Actuellement, vous n'avez pas encore chargé les données de MovieLens localement pour les examiner. Exécutez les lignes ci-dessous pour télécharger la dernière copie et l'examiner rapidement.

#### Télécharger et explorer le jeu de données

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### Préparer et charger les données

Comme vous pouvez le voir, les données contiennent un UserID, un ItemID, un Rating et un Timestamp.

Nous allons maintenant retirer les articles ayant un faible classement, et supprimer la colonne Classement avant de créer notre modèle.

Ensuite, nous allons maintenant enregistrer le fichier comme un nouveau fichier CSV et le télécharger vers S3.

Pour ce faire, nous allons exécuter les lignes dans la cellule ci-dessous.

In [ ]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Créer un schéma

L'un des principaux articles permettant à Personalize de comprendre vos données provient du schéma défini ci-dessous. Cette configuration indique au service comment traiter les données fournies par votre fichier CSV. Notez que les colonnes et les types s'alignent sur les articles présents dans le fichier que vous avez créé ci-dessus.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### Créer et attendre un groupe de jeux de données

Le groupe le plus important dans Personalize est le groupe de jeu de données, qui permet d'isoler vos données, vos traceurs d'événements, vos solutions et vos campagnes. Regroupement de choses qui partagent un ensemble commun de données. N'hésitez pas à modifier le nom ci-dessous si vous le souhaitez.

#### Créer un groupe de jeux de données

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-launch-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Attendre que le groupe de jeu de données ait le statut ACTIF

Avant de pouvoir utiliser le groupe de jeu de données dans les articles ci-dessous, il doit être actif; exécutez la cellule ci-dessous et attendez qu'il soit actif.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Créer un jeu de données

Après le groupe, vous devez créer les jeux de données réels. Dans cet exemple, nous allons en créer un seul pour les données d'interactions. Exécutez les cellules ci-dessous pour le créer.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-launch-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### Attacher une politique au compartiment S3

Amazon Personalize doit pouvoir lire le contenu du compartiment S3 que vous avez créé précédemment. Les lignes ci-dessous le permettent.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

#### Créer un rôle Personalize

De plus, Amazon Personalize doit pouvoir assumer des rôles dans AWS, afin d'avoir les autorisations d'exécuter certaines tâches. Les lignes ci-dessous les accordent.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Importer les données

Vous avez précédemment créé le groupe de jeu de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données de S3 dans Amazon Personalize pour les utiliser dans la création de votre modèle.

#### Créer une tâche d'importation de jeu de données

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import1",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### Attendre que la tâche d'importation de données ait le statut ACTIF

La tâche d'importation peut durer un certain temps ; veuillez patienter jusqu'à ce que vous voyiez qu'elle est active ci-dessous.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Créer la solution et la version

Dans Amazon Personalize, un modèle entraîné est appelé Solution. Chaque Solution peut avoir plusieurs versions spécifiques qui se rapportent à un volume de données donné lorsque le modèle a été formé.

Pour commencer, nous allons énumérer toutes les recettes qui sont prises en charge. Une recette est un algorithme qui n'a pas encore été entraîné sur vos données. Après l'énumération, vous en choisirez une et l'utiliserez pour créer votre modèle.

### Sélectionner une recette

In [ ]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

#### Personnalisation de l'utilisateur
La recette [Personnalisation utilisateur](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html) (aws-user-personalization) est optimisée pour tous les scénarios de recommandation USER_PERSONALIZATION. Lorsque vous recommandez des articles, elle utilise l'exploration automatique des articles.

Avec l'exploration automatique, Amazon Personalize teste automatiquement les différentes recommandations d'articles, découvre comment les utilisateurs interagissent avec ces articles recommandés et stimule les recommandations pour les articles qui favorisent un meilleur engagement et une meilleure conversion. Cela améliore la découverte des articles et l'engagement lorsque vous disposez d'un catalogue qui évolue rapidement, ou lorsque de nouveaux articles, tels que des articles d'actualité ou des promotions, sont plus pertinents pour les utilisateurs lorsqu'ils sont frais.

Vous pouvez équilibrer le niveau d'exploration (où les articles présentant moins de données d'interaction ou de pertinence sont recommandés plus fréquemment) et le niveau d'exploitation (où les recommandations sont basées sur ce que nous savons ou sur la pertinence). Amazon Personalize ajuste automatiquement les recommandations futures en fonction des retours implicites des utilisateurs.

Tout d'abord, sélectionnez la recette en trouvant l'ARN dans la liste des recettes ci-dessus.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### Créer et attendre la solution

Vous créez d'abord la solution avec l'API, puis vous créez une version. L'entraînement de votre modèle pour créer votre version de la solution prendra plusieurs minutes. Une fois qu'il a commencé et que vous voyez les notifications en cours, c'est le moment de faire une pause, de prendre un café, etc.

#### Créer une solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-user-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Créer une version de solution

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### Attendre que la version de la solution ait le statut ACTIVE (ACTIF)

Cela prendra environ 40 à 50 minutes.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Obtenir les métriques sur la version de la solution

Maintenant que votre solution et sa version sont en place, vous pouvez obtenir les métriques qui vous permettront de juger de ses performances. Ces métriques ne sont pas particulièrement bonnes, car il s'agit d'un jeu de données de démonstration. Cependant, vous vous devriez constater des améliorations avec des jeux de données plus grands et plus complexes.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

Nous recommandons de lire [la documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) pour comprendre les métriques, mais nous avons également copié des parties de la documentation ci-dessous pour plus de commodité.

Vous devez comprendre les termes suivants à propos de l'évaluation dans Personalize :

- *Relevant recommendation* est une recommandation qui correspond à une valeur dans les données de test pour un utilisateur particulier.
- *Rank* fait référence à la position d'un article recommandé dans la liste des recommandations. La position 1 (haut de la liste) est présumée être la plus pertinente pour l'utilisateur.
- *Query* fait référence à l'équivalent interne d'un appel GetRecommendations.

Les métriques fournies par Personalize sont les suivantes :

- coverage : La proportion d'articles uniques recommandés à partir de toutes les requêtes sur le nombre total d'articles uniques dans les données de formation (comprend à la fois les jeux de données d'articles et d'interactions).
- mean_reciprocal_rank_at_25 : La [moyenne des rangs réciproques](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) de la première recommandation pertinente parmi les 25 premières recommandations pour toutes les requêtes. Cette métrique est appropriée si vous êtes intéressé par la recommandation unique la mieux classée.
- normalized_discounted_cumulative_gain_at_K : Le gain actualisé suppose que les recommandations situées plus bas dans une liste de recommandations sont moins pertinentes que les recommandations situées plus haut. Par conséquent, chaque recommandation est réduite (pondération plus faible) par un facteur dépendant de sa position. Pour produire le [gain cumulé actualisé](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) à K, toutes les recommandations actualisées pertinentes dans les K premières recommandations sont additionnées. Le gain cumulé actualisé normalisé (NDCG) est le DCG divisé par le DCG idéal de sorte que le NDCG soit compris entre 0 et 1. (Le DCG idéal est celui où les K premières recommandations sont triées par pertinence). Amazon Personalize utilise un facteur de pondération de 1/log(1 + position), où le haut de la liste est la position 1. Cette métrique récompense les articles pertinents qui apparaissent près du haut de la liste, car le haut d'une liste suscite généralement plus d'attention.
- precision_at_K : Le nombre de recommandations pertinentes parmi les K premières recommandations divisé par K. Cette métrique récompense la recommandation précise des articles pertinents.

## Créer et attendre la campagne

Maintenant que vous avez une version de la solution opérationnelle, vous devez créer une campagne pour l'utiliser avec vos applications. Une campagne est une version de la solution hébergée, un point de terminaison que vous pouvez interroger pour obtenir des recommandations. La tarification est fixée en estimant la capacité de débit (demandes de personnalisation de l'utilisateur par seconde). Lors du déploiement d'une campagne, vous définissez une valeur minimale de transactions par seconde (TPS) (`minProvisionedTPS`). Ce service, comme beaucoup d'autres au sein d'AWS, évoluera automatiquement en fonction de la demande. Néanmoins, si la latence est stratégique, vous voudrez peut-être prendre des dispositions pour une demande plus importante. Pour cette démo, le seuil de débit minimum est défini à 1. Pour plus d'informations, reportez-vous à la page de [tarification](https://aws.amazon.com/personalize/pricing/).

Comme mentionné ci-dessus, la recette de personnalisation de l'utilisateur utilisée pour notre solution prend en charge l'exploration automatique des articles "cold". Vous pouvez contrôler le niveau d'exploration lors de la création de votre campagne. Le type de données `itemExplorationConfig` prend en charge les paramètres `explorationWeight` et `explorationItemAgeCutOff`. La pondération de l'exploration détermine la fréquence à laquelle les recommandations incluent les articles dont les données d'interaction ou la pertinence sont moindres. Plus la valeur est proche de 1,0, plus l'exploration est forte. À zéro, aucune exploration n'a lieu et les recommandations sont basées sur les données actuelles (pertinence). Le critère d'âge des articles à explorer détermine les articles à explorer en fonction du temps écoulé depuis la dernière interaction. Fournir l'âge maximal de l'article, en jours depuis la dernière interaction, pour définir la portée de l'exploration de l'article. Plus la valeur est élevée, plus le nombre d'articles pris en compte lors de l'exploration est élevé. Pour notre campagne ci-dessous, nous allons spécifier une pondération d'exploration de 0,5.

#### Créer une campagne

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-demo-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.5"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Attendre que la campagne ait le statut ACTIF

Cela devrait prendre environ 10 minutes.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Obtenir des exemples de recommandations

Une fois la campagne active, vous êtes prêt à recevoir des recommandations. Tout d'abord, nous devons sélectionner un utilisateur aléatoire à partir de la collection. Ensuite, nous créerons quelques fonctions d'aide pour obtenir des informations sur les films à afficher pour les recommandations au lieu de simples ID.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE']


#### Appeler GetRecommendations

En utilisant l'utilisateur que vous avez obtenu ci-dessus, les lignes ci-dessous obtiendront des recommandations pour vous et retourneront la liste des films recommandés.


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs'])
recommendations_df

## Examiner

En utilisant les codes ci-dessus, vous avez entraîné un modèle de deep learning pour générer des recommandations de films en fonction du comportement antérieur de l'utilisateur. Réfléchissez à d'autres types de problèmes pour lesquels ces données sont disponibles et à quoi pourrait ressembler la création d'un système comme celui-ci pour offrir ces recommandations.

Vous êtes maintenant prêt à passer au bloc-notes suivant `2.View_Campaign_And_Interactions.ipynb`.



## Remarques pour le bloc-notes suivant :

Vous aurez besoin de quelques valeurs pour le prochain bloc-notes. Exécutez les cellules ci-dessous pour les stocker, afin qu'elles puissent être copiées et collées dans la partie suivante de l'exercice.

In [ ]:
%store campaign_arn

In [ ]:
%store dataset_group_arn

In [ ]:
%store solution_version_arn

In [ ]:
%store solution_arn

In [ ]:
%store dataset_arn

In [ ]:
%store campaign_arn

In [ ]:
%store schema_arn

In [ ]:
%store bucket

In [ ]:
%store filename

In [ ]:
%store role_name

In [ ]:
%store recommendations_df

In [ ]:
%store user_id